In [2]:
import pandas as pd
import numpy as np
from joblib import Memory
import time
import os
import keras
from TINTOlib.tinto import TINTO
from tensorflow_addons.metrics import RSquare

c:\Users\Medion\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
# Cargamos el dataset
df = pd.read_csv("C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/Data/ULA_8.csv")

In [4]:
print(df.shape)
print(df.iloc[:2,:])

(252004, 1602)
   Antena1Portadora1Modulo  Antena1Portadora1Angulo  Antena1Portadora2Modulo  \
0                 0.601309                -0.428778                 0.620221   
1                 0.436977                -0.835988                 0.447176   

   Antena1Portadora2Angulo  Antena1Portadora3Modulo  Antena1Portadora3Angulo  \
0                -0.449564                 0.626025                -0.508311   
1                -0.890598                 0.457048                -0.918748   

   Antena1Portadora4Modulo  Antena1Portadora4Angulo  Antena1Portadora5Modulo  \
0                 0.641815                -0.550858                 0.642635   
1                 0.475890                -0.948638                 0.477857   

   Antena1Portadora5Angulo  ...  Antena8Portadora97Modulo  \
0                -0.564398  ...                  0.859801   
1                -1.000932  ...                  0.630989   

   Antena8Portadora97Angulo  Antena8Portadora98Modulo  \
0                 -0.

In [5]:
# Fila que queremos predecir (falta añadirle ruido)
data = df.loc[(df['PosicionX'] == -1052) & (df['PosicionY'] == 2110)]
print(data)

        Antena1Portadora1Modulo  Antena1Portadora1Angulo  \
246511                 0.178537                -2.965653   

        Antena1Portadora2Modulo  Antena1Portadora2Angulo  \
246511                 0.162474                -2.972491   

        Antena1Portadora3Modulo  Antena1Portadora3Angulo  \
246511                 0.174036                -2.983824   

        Antena1Portadora4Modulo  Antena1Portadora4Angulo  \
246511                 0.177337                -3.009041   

        Antena1Portadora5Modulo  Antena1Portadora5Angulo  ...  \
246511                 0.171919                -3.118869  ...   

        Antena8Portadora97Modulo  Antena8Portadora97Angulo  \
246511                  0.727119                  2.574451   

        Antena8Portadora98Modulo  Antena8Portadora98Angulo  \
246511                  0.728335                  2.581856   

        Antena8Portadora99Modulo  Antena8Portadora99Angulo  \
246511                  0.746676                  2.633478   

        An

In [6]:
df_copia = df.copy()

In [10]:
# * Normalizamos los datos
df = df.head(100) # Limitamos el dataset a 100 filas para que no tarde tanto
columns_to_normalize = df.columns[:-2]
print(columns_to_normalize)
df_normalized = (df[columns_to_normalize] - df[columns_to_normalize].min()) / (df[columns_to_normalize].max() - df[columns_to_normalize].min())
print("-------")
print(df_normalized)
df_normalized = pd.concat([df_normalized, df[df.columns[-2]], df[df.columns[-1]]], axis=1)
print("-------")
print(df_normalized)

Index(['Antena1Portadora1Modulo', 'Antena1Portadora1Angulo',
       'Antena1Portadora2Modulo', 'Antena1Portadora2Angulo',
       'Antena1Portadora3Modulo', 'Antena1Portadora3Angulo',
       'Antena1Portadora4Modulo', 'Antena1Portadora4Angulo',
       'Antena1Portadora5Modulo', 'Antena1Portadora5Angulo',
       ...
       'Antena8Portadora96Modulo', 'Antena8Portadora96Angulo',
       'Antena8Portadora97Modulo', 'Antena8Portadora97Angulo',
       'Antena8Portadora98Modulo', 'Antena8Portadora98Angulo',
       'Antena8Portadora99Modulo', 'Antena8Portadora99Angulo',
       'Antena8Portadora100Modulo', 'Antena8Portadora100Angulo'],
      dtype='object', length=1600)
-------
    Antena1Portadora1Modulo  Antena1Portadora1Angulo  Antena1Portadora2Modulo  \
0                  0.662441                 0.430854                 0.681627   
1                  0.445998                 0.364748                 0.467091   
2                  0.496891                 0.450336                 0.514340   

In [116]:
# Las imágenes se generan con el df. (no el normalizado)
image_folder = "C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/Images/Jupyter"
image_model = TINTO(problem = "regression", pixels = 35, blur = True)

if not os.path.exists(image_folder):
    print("Generating images")
    image_model.generateImages(df.iloc[:,:-1], image_folder)
    
img_paths = os.path.join(image_folder + "/regression.csv")

imgs = pd.read_csv(img_paths)
print(imgs.shape)
print(imgs.iloc[:5,:])

(100, 2)
              images  values
0  images\000000.png   302.0
1  images\000001.png   722.0
2  images\000002.png  1117.0
3  images\000003.png  -892.0
4  images\000004.png -1217.0


In [117]:
imgs["images"] = image_folder + "/" + imgs["images"]
imgs["images"] = imgs["images"].str.replace("\\","/")
# print(imgs["images"]) #   C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/Images/Jupyter/images/000000.png

# print("-------")
img_1 = imgs.iloc[-1:,:]
print(img_1["images"])

data_1 = df_normalized.iloc[-1:,:]
#print(data_1.shape)
# print("-------")
# print(data_1)

combined_data_img = pd.concat([img_1, data_1.iloc[:,:-1]], axis=1)
# print("-----")
# print(combined_data_img)

99    C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I...
Name: images, dtype: object


In [45]:
combined_dataset_x = pd.concat([imgs,df_normalized.iloc[:,:-1]],axis=1)
print(combined_dataset_x.shape)
print(combined_dataset_x.iloc[:5,:]) # images, values, datos, posicionX

print("------")

combined_dataset_y = pd.concat([imgs,pd.concat([df_normalized.iloc[:,:-2], df_normalized.iloc[:,-1:]],axis=1)],axis=1)
print(combined_dataset_y.shape)
print(combined_dataset_y.iloc[:5,:]) # images, values, datos, posicionY

(100, 1603)
                                              images  values  \
0  C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I...   302.0   
1  C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I...   722.0   
2  C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I...  1117.0   
3  C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I...  -892.0   
4  C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I... -1217.0   

   Antena1Portadora1Modulo  Antena1Portadora1Angulo  Antena1Portadora2Modulo  \
0                 0.662441                 0.430854                 0.681627   
1                 0.445998                 0.364748                 0.467091   
2                 0.496891                 0.450336                 0.514340   
3                 0.242174                 0.363422                 0.292943   
4                 0.285156                 0.904383                 0.301483   

   Antena1Portadora2Angulo  Antena1Portadora3Modulo  Antena1Portadora3Angulo  \
0                 0.431355                

In [32]:
df_x = combined_dataset_x.drop("PosicionX",axis=1).drop("values",axis=1) # Solo imagenes y datos
print(df_x.iloc[:1,:])

print("------")

df_y_for_x = combined_dataset_x["values"]
df_y_for_y = combined_dataset_y["PosicionY"]

print(df_y_for_x.iloc[:6]) # Posiciones de X
print("------")
print(df_y_for_y.iloc[:6]) # Posiciones de Y

                                              images  Antena1Portadora1Modulo  \
0  C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I...                 0.662441   

   Antena1Portadora1Angulo  Antena1Portadora2Modulo  Antena1Portadora2Angulo  \
0                 0.430854                 0.681627                 0.431355   

   Antena1Portadora3Modulo  Antena1Portadora3Angulo  Antena1Portadora4Modulo  \
0                 0.673291                 0.417094                 0.675869   

   Antena1Portadora4Angulo  Antena1Portadora5Modulo  ...  \
0                 0.409565                 0.693136  ...   

   Antena8Portadora96Modulo  Antena8Portadora96Angulo  \
0                   0.52171                  0.453752   

   Antena8Portadora97Modulo  Antena8Portadora97Angulo  \
0                  0.536045                  0.449234   

   Antena8Portadora98Modulo  Antena8Portadora98Angulo  \
0                  0.513473                  0.447106   

   Antena8Portadora99Modulo  Antena8Portadora99An

In [35]:
# Mezclamos las filas

np.random.seed(64)
df_x = df_x.sample(frac=1).reset_index(drop=True) # 

np.random.seed(64)
df_y_for_x = df_y_for_x.sample(frac=1).reset_index(drop=True)

np.random.seed(64)
df_y_for_y = df_y_for_y.sample(frac=1).reset_index(drop=True)


In [38]:
trainings_size = 0.85                     # 85% training set
validation_size = 0.1                     # 10% validation set
test_size = 0.05                         # 5% test set

X_train = df_x.iloc[:int(trainings_size*len(df_x))]
print(X_train.iloc[:1,:])
y_train_x = df_y_for_x.iloc[:int(trainings_size*len(df_y_for_x))]
y_train_y = df_y_for_y.iloc[:int(trainings_size*len(df_y_for_y))]

X_val = df_x.iloc[int(trainings_size*len(df_x)):int((trainings_size+validation_size)*len(df_x))]
y_val_x = df_y_for_x.iloc[int(trainings_size*len(df_y_for_x)):int((trainings_size+validation_size)*len(df_y_for_x))]
y_val_y = df_y_for_y.iloc[int(trainings_size*len(df_y_for_y)):int((trainings_size+validation_size)*len(df_y_for_y))]

X_test = df_x.iloc[-int(test_size*len(df_x)):]
y_test_x = df_y_for_x.iloc[-int(test_size*len(df_y_for_x)):]
y_test_y = df_y_for_y.iloc[-int(test_size*len(df_y_for_y)):]

X_train_num = X_train.drop("images",axis=1)
X_val_num = X_val.drop("images",axis=1)
X_test_num = X_test.drop("images",axis=1)

                                              images  Antena1Portadora1Modulo  \
0  C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I...                 0.240925   

   Antena1Portadora1Angulo  Antena1Portadora2Modulo  Antena1Portadora2Angulo  \
0                 0.498206                 0.272242                 0.490647   

   Antena1Portadora3Modulo  Antena1Portadora3Angulo  Antena1Portadora4Modulo  \
0                 0.240282                 0.489603                 0.251894   

   Antena1Portadora4Angulo  Antena1Portadora5Modulo  ...  \
0                 0.476454                 0.265267  ...   

   Antena8Portadora96Modulo  Antena8Portadora96Angulo  \
0                  0.082019                  0.187109   

   Antena8Portadora97Modulo  Antena8Portadora97Angulo  \
0                  0.099532                  0.183592   

   Antena8Portadora98Modulo  Antena8Portadora98Angulo  \
0                  0.114284                  0.179016   

   Antena8Portadora99Modulo  Antena8Portadora99An

In [119]:
import cv2
pixel = 35

X_train_img = np.array([cv2.resize(cv2.imread(img),(pixel,pixel)) for img in X_train["images"]])
X_val_img = np.array([cv2.resize(cv2.imread(img),(pixel,pixel)) for img in X_val["images"]])
X_test_img = np.array([cv2.resize(cv2.imread(img),(pixel,pixel)) for img in X_test["images"]])

# x_test_img_1 = np.array([cv2.resize(cv2.imread(img),(pixel,pixel)) for img in X_test["images"].iloc[0]])
# x_test_img_1 = np.array([cv2.resize(cv2.imread(img_1["images"]),(pixel,pixel))])
# x_test_img_1 = np.array([cv2.resize(cv2.imread(X_test["images"].iloc[:1]),(pixel,pixel))])

# route = img_1["images"]
# if isinstance(route, str):
#     print("AA")
# route_2 = X_test["images"].iloc[0]
# print(route_2)
# if isinstance(route_2, str):
#     print("BB")
    
# for img in X_test["images"].iloc[0]:
#     if isinstance(img, str):
#         print("CC")
#         break

# route = str(route)
# img = cv2.resize(cv2.imread(route), (pixel,pixel))
# print(img)

image_path = img_1["images"].iloc[0]
print(image_path)
image = cv2.imread(image_path)

if image is not None:
    # Continuar con el procesamiento de la imagen redimensionada
    try:
        resized_image = cv2.resize(image, (pixel, pixel))
        print(resized_image)
        print("---")
        print(np.array(resized_image))
        # Continuar con el procesamiento de la imagen redimensionada
    except Exception as e:
        print("Error al redimensionar la imagen:", e)
else:
    print("La imagen no se cargó correctamente.")


validacion_x = y_val_x
test_x = y_test_x
validacion_y = y_val_y
test_y = y_test_y

shape = len(X_train_num.columns) #1600

C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/Images/Jupyter/images/000099.png
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
---
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255

In [44]:
#predict([X_test_num, X_test_img])
print(X_test_num)
print("------")
print(X_test_img)

    Antena1Portadora1Modulo  Antena1Portadora1Angulo  Antena1Portadora2Modulo  \
95                 0.165156                 0.964502                 0.190093   
96                 0.181244                 0.045244                 0.232176   
97                 0.005502                 0.805736                 0.061883   
98                 0.063578                 0.127140                 0.128677   
99                 0.379075                 0.011871                 0.388504   

    Antena1Portadora2Angulo  Antena1Portadora3Modulo  Antena1Portadora3Angulo  \
95                 0.963395                 0.198283                 0.965993   
96                 0.037792                 0.234236                 0.041232   
97                 0.793160                 0.005682                 0.803624   
98                 0.132361                 0.064057                 0.105543   
99                 0.009860                 0.402047                 0.000000   

    Antena1Portadora4Modul

In [120]:
x = [-945.6397]
print(x[0])

-945.6397
